In [1]:
# https://developers.google.com/optimization/introduction/python
# to do: 
# all or nothing flow - maximum one inflow and one outflow only
# prune variables for the model (so that it is not 100k variables)
# plotting - reduce clutter

In [2]:
%reset -sf
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from ortools.linear_solver import pywraplp

# Define problem

In [3]:
df = pd.read_pickle("../preprocessing/processed_dataframe.pkl")
# df

In [4]:
nodes = pd.DataFrame()
nodes["supplies"] = df["TOTAL"]
nodes["x"] = df["x_centre"]
nodes["y"] = df["y_centre"]

start_nodes = []
end_nodes   = []
unit_costs  = []
for i,row in df.iterrows():
    for adj, cst in zip(row["adjacent"], row["shared_param"]):
        start_nodes.append(i)
        end_nodes.append(adj)
        unit_costs.append(cst)

edges = pd.DataFrame()
edges["start_nodes"] = start_nodes
edges["end_nodes"]   = end_nodes
edges["unit_costs"]  = unit_costs
edges["capacities"]  = [9999999]*len(start_nodes) # disabled
edges["flows"]       = [0]*len(start_nodes)
edges["flowing"]     = [0]*len(start_nodes)
edges["names"]       = "x"+edges[["start_nodes",
                                  "end_nodes"]].astype(str).apply('-'.join, axis=1)
edges.index = edges["names"]
N = len(nodes) # the number of nodes at init

In [ ]:
edges.sample(5)

,start_nodes,end_nodes,unit_costs,capacities,flows,flowing,names
names,,,,,,,
x184-172,184,172,0.004475,9999999,0,0,x184-172
x190-221,190,221,0.014177,9999999,0,0,x190-221
x260-310,260,310,0.017209,9999999,0,0,x260-310
x303-294,303,294,0.005171,9999999,0,0,x303-294
x45-48,45,48,0.015239,9999999,0,0,x45-48


# Augment nodes to a common sink

In [ ]:
def augment_nodes(nodes, edges):
    for i,ss in enumerate(nodes["supplies"]):
        edges.loc[len(edges)] = [i, len(nodes), 0, 9999999, 0, 0,
                                 "x"+str(i)+"-"+str(len(nodes))]
    nodes.loc[len(nodes)] = [-sum(nodes["supplies"]), 103.5, 1.2]
    edges.index = edges["names"]
    return nodes, edges

In [ ]:
nodes, edges = augment_nodes(nodes, edges)
# nodes
# edges

# Generate adjacency matrix

In [ ]:
def get_adjacency_matrix(edges):
    start_nodes = edges["start_nodes"]
    end_nodes = edges["end_nodes"]
    unit_costs = edges["unit_costs"]
    capacities = edges["capacities"]
    flows = edges["flows"]
    flowing = edges["flowing"]
    names = edges["names"]
    
    matrix_unit_costs = [[0 for _ in range(N+1)] for _ in range(N+1)]
    matrix_capacities = [[0 for _ in range(N+1)] for _ in range(N+1)]
    matrix_flows      = [[0 for _ in range(N+1)] for _ in range(N+1)]
    matrix_flowing    = [[0 for _ in range(N+1)] for _ in range(N+1)]
    matrix_names      = [["x"+str(i)+"-"+str(j)
                          for i in range(N+1)] 
                         for j in range(N+1)]
    
    for x,y,z,c,f,g in zip(start_nodes, end_nodes, unit_costs, capacities, flows, flowing):
        matrix_unit_costs[x][y] = z
        matrix_capacities[x][y] = c
        matrix_flows     [x][y] = f
        matrix_flowing   [x][y] = g
        
    for y,x,z,c,f,g in zip(start_nodes, end_nodes, unit_costs, capacities, flows, flowing):
        matrix_unit_costs[x][y] = z
        matrix_capacities[x][y] = c
        matrix_flows     [x][y] = f
        matrix_flowing   [x][y] = g

    return matrix_unit_costs, matrix_capacities, matrix_flows, matrix_names, matrix_flowing

(matrix_unit_costs, matrix_capacities, 
 matrix_flows, matrix_names, matrix_flowing) = get_adjacency_matrix(edges)

In [ ]:
# matrix_unit_costs, matrix_capacities, matrix_flows, matrix_names

# Visualisation

In [ ]:
edges.sample(5)

,start_nodes,end_nodes,unit_costs,capacities,flows,flowing,names
names,,,,,,,
x218-229,218,229,0.004190,9999999,0,0,x218-229
x187-56,187,56,0.011054,9999999,0,0,x187-56
x58-125,58,125,0.023051,9999999,0,0,x58-125
x33-31,33,31,0.002571,9999999,0,0,x33-31
x58-197,58,197,0.015183,9999999,0,0,x58-197


In [ ]:
def plot_graph(nodes, edges):
    start_nodes = edges["start_nodes"]
    end_nodes = edges["end_nodes"]
    unit_costs = edges["unit_costs"]
    capacities = edges["capacities"]
    flows = edges["flows"]
    names = edges["names"]

    G = nx.DiGraph()
    for x,y,z,c,f,n in zip(start_nodes, end_nodes, unit_costs, capacities, flows, names):
        G.add_edge(x, y, cost=z, capacity=c, flow=f, name=n)

    labeldict = {}
    for i,s in enumerate(nodes["supplies"]):
        labeldict[i] = s

    fig, ax = plt.subplots(figsize=(14,9))
    
    pos=nx.circular_layout(G)
    pos={}
    for i,node in nodes.iterrows():
        pos[i] = [node["x"], node["y"]]
    
    for edge in G.edges(data=True):
        w = edge[2]['flow']
        if w > 0:
            size = np.log(1+w)
            nx.draw_networkx_edge_labels(G,pos, edgelist=[(edge[0],edge[1])], 
                                         edge_labels = {(edge[0],edge[1]): w})
            nx.draw_networkx_edges(G, pos, edgelist=[(edge[0],edge[1])], 
                                   width=size, arrowsize=size*10)
            
    nx.draw_networkx_nodes(G, pos, with_labels=True, font_size=15, 
                           node_size=200, 
                           node_color="yellow")
    nx.draw_networkx_labels(G, pos, labels=labeldict)
    
    ax.autoscale()
    ax.set_aspect(1)
    ax.margins(0.1)
    plt.show()

In [ ]:
plt.show()
plot_graph(nodes, edges)

# Min-cost flow as a linear program

In [ ]:
solver = pywraplp.Solver('hello_program',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# define variables and capacities
infinity = solver.infinity()
X = [[solver.NumVar(0.0, matrix_capacities[i][j], 'x'+str(i)+"-"+str(j)) 
      for j in range(N+1)]
     for i in range(N+1)]

# define supplies
for i in range(N+1):
    solver.Add(sum([X[i][j] for j in range(N+1)])
             - sum([X[j][i] for j in range(N+1)])
            == nodes["supplies"][i])

In [ ]:
B = [[solver.IntVar(0.0, 1.0, 'b'+str(i)+"-"+str(j)) 
      for j in range(N+1)]
     for i in range(N+1)]

In [ ]:
%%time
for i in range(N+1):
    for j in range(N+1):
        if matrix_capacities[i][j] > 0:
            solver.Add(X[i][j] <= 1200000*B[i][j])
    solver.Add(sum([B[i][j] for j in range(N+1)]) <= 1)

In [ ]:
# for i in range(N):
#     solver.Add(X[i][N] <= 1200000*B[i][N])
#     solver.Add(X[N][i] <= 1200000*B[N][i])
#     solver.Add(sum([B[i][j] for j in range(N)]) <= 1)

In [ ]:
# limit number of cstcs
# solver.Add(sum([B[i][N] for i in range(N)]) <= 10) # what I want to do
solver.Add(B[N-1][N] + B[N-2][N]+ B[N-3][N] + B[N-4][N] + B[N-5][N] <= 1) # works
# solver.Add(sum(B[N-1][N] for i in range(5)) <= 1) # does not work
# https://groups.google.com/forum/#!topic/or-tools-discuss/UQraf8LYJ14 ??

In [ ]:
# for i in range(N):
#     print(X[N][i].solution_value() for j in range(N)]))

In [ ]:
# define objective
solver.Minimize(sum([sum([matrix_unit_costs[i][j] * X[i][j]
                          for j in range(N+1)])
                     for i in range(N+1)]))

solver.set_time_limit(10*1000)

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

In [ ]:
# print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

In [ ]:
status = solver.Solve()

# Visualise output

In [ ]:
print('NOT_SOLVED: ', status == pywraplp.Solver.NOT_SOLVED)
print('Stopped by time limit: ', status == pywraplp.Solver.FEASIBLE)
print('Optimal: ', status == pywraplp.Solver.OPTIMAL)
print('Infeasible: ', status == pywraplp.Solver.INFEASIBLE)

In [ ]:
print('Solution:')
print('Objective value =', solver.Objective().Value())
print('')
print('Advanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations'   % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

In [ ]:
matrix_flows = [[x.solution_value() for x in xxx] for xxx in X]
matrix_flowing = [[b.solution_value() for b in bbb] for bbb in B]
for i,row in enumerate(matrix_flows):
    for j,cell in enumerate(row):
        if matrix_flows[i][j] > 0:
            edges.at["x"+str(i)+"-"+str(j),"flows"] = matrix_flows[i][j]
            edges.at["x"+str(i)+"-"+str(j),"flowing"] = matrix_flowing[i][j]

In [ ]:
plot_graph(nodes, edges)

In [ ]:
edges.sample(5)

In [ ]:
edges.tail(5)

In [ ]:
(  B[N-1][N].solution_value() 
 + B[N-2][N].solution_value() 
 + B[N-3][N].solution_value() 
 + B[N-4][N].solution_value() 
 + B[N-5][N].solution_value())

In [ ]:
edges[edges["flows"]>0].sort_index().sort_values(by="start_nodes").sort_values(by="end_nodes").tail(100)

In [ ]:
arr = [sum([r > 0 for r in row]) for row in np.transpose(matrix_flows)][:-1]
print(sum(arr))
plt.plot(arr)
plt.show()

In [ ]:
arr = [sum(row) for row in np.transpose(matrix_flowing)][:-1]
print(sum(arr))
plt.plot(arr)
plt.show()

In [ ]:
arr = [sum(row) for row in matrix_flowing]
print(sum(arr))
plt.plot(arr)
plt.show()

In [ ]:
arr = [sum([r > 0 for r in row]) for row in matrix_flows]
print(sum(arr))
plt.plot(arr)
plt.show()

In [ ]:
index = [sum([r > 0 for r in row]) for row in matrix_flows].index(2)
index

In [ ]:
edges[edges["end_nodes"] == index]

In [ ]:
edges[edges["start_nodes"] == index]

In [ ]:
nodes.loc[index]

In [ ]:
sum(nodes["supplies"])